In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient()

In [3]:
coll = client.companies.companies

In [4]:
coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies')

## Ex. 20: Get all the companies that were deadpooled after the 3rd year.

### Aggregation

We can use an aggregation query to `project` (show, represent, "create" a new value, that subtracts the  founding year from the deadpooling year.

We can also `match` or filter only those where the newly created column "deadpool" is larger than 3.

In [5]:
res = coll.aggregate(
    [
        {"$project":{"name":1, "founded_year":1, "deadpooled_year":1, 
                     "deadpool":{"$subtract":["$deadpooled_year","$founded_year"]}}},
        {"$match":{"deadpool":{"$gt":3}}}
    ]
)

In [6]:
res=list(res)
res[:3]

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'),
  'name': 'Babelgum',
  'founded_year': 2007,
  'deadpooled_year': 2013,
  'deadpool': 6},
 {'_id': ObjectId('52cdef7c4bab8bd675297dae'),
  'name': 'Thoof',
  'founded_year': 2006,
  'deadpooled_year': 2013,
  'deadpool': 7},
 {'_id': ObjectId('52cdef7c4bab8bd675297db6'),
  'name': 'Wesabe',
  'founded_year': 2005,
  'deadpooled_year': 2010,
  'deadpool': 5}]

### \$where

Alternatively, we can use the `$where` operator, which allows us to pass a JavaScript expression or operation to it. 

- `Note:`  We use `this` to refer to the `self` of the document. This way we can access all of their attributes.

In [7]:
res = coll.find(
    {
        "$where":
        "(this.deadpooled_year - this.founded_year > 3)&(this.founded_year !== null)"
        # Checking that is was deadpooled after 3 years and that there are no null founded years.
    },  # The filter
    {"name":1,"founded_year":1,"deadpooled_year":1} # The project
)

In [8]:
res = list(res)
res[:3]

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'),
  'name': 'Babelgum',
  'founded_year': 2007,
  'deadpooled_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd675297dae'),
  'name': 'Thoof',
  'founded_year': 2006,
  'deadpooled_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd675297db6'),
  'name': 'Wesabe',
  'founded_year': 2005,
  'deadpooled_year': 2010}]